In [ ]:
%%writefile bfs_openmp.cpp
#include <iostream>
#include <queue>
#include <omp.h>
using namespace std;

const int MAX = 1000;
int graph[MAX][MAX], visited[MAX];

void bfs(int start, int n) {
    queue<int> q;
    visited[start] = 1;
    q.push(start);

    while (!q.empty()) {
        int curr = q.front();
        q.pop();

        // Parallelize visiting neighbors
        #pragma omp parallel for shared(graph, visited, q) schedule(dynamic)
        for (int i = 0; i < n; i++) {
            if (graph[curr][i]) {
                // Use critical section to avoid race conditions
                #pragma omp critical
                {
                    if (!visited[i]) {
                        visited[i] = 1;
                        q.push(i);
                    }
                }
            }
        }
    }
}

int main() {
    int n, start;
    cout << "Enter number of vertices: ";
    cin >> n;

    cout << "Enter adjacency matrix:\n";
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            cin >> graph[i][j];
        }
    }

    cout << "Enter starting vertex: ";
    cin >> start;

    // Initialize visited array to 0
    for (int i = 0; i < n; i++) {
        visited[i] = 0;
    }

    bfs(start, n);

    cout << "BFS traversal: ";
    for (int i = 0; i < n; i++) {
        if (visited[i])
            cout << i << " ";
    }
    cout << endl;
    return 0;
}


Writing bfs_openmp.cpp


In [ ]:
!g++ -fopenmp bfs_openmp.cpp -o bfs_openmp

In [ ]:
!./bfs_openmp

Enter number of vertices: 5
Enter adjacency matrix:
0 1 0 1 0 
0 1 1 1 1 
1 0 0 0 1 
1 0 1 0 1 
1 0 0 0 0 
Enter starting vertex: 0
BFS traversal: 0 1 2 3 4 
